In [ ]:
import pandas as pd
import re
from collections import Counter

**Loading the data from the 1884 OCR**

In [ ]:
df_dict = pd.read_excel("beau_monde_1884_tables.xlsx",header = None, sheet_name = None)

In [ ]:
df_dict['table_1'].head()

How many rows with missing values (or nan values) do we have?

In [ ]:
counter = 0
for table in df_dict:
    counter += df_dict[table].isna().sum().sum()
print('There are %d missing values in our data.' %(counter))

Since there a too many values to fill them in by hand, we will just get rid of the corresponding rows (at least for now)

In [ ]:
for table in df_dict:
    df_dict[table].dropna(inplace=True)
    
counter = 0
for table in df_dict:
    counter += df_dict[table].isna().sum().sum()
print('Now, there are %d missing values in our data.' %(counter))

Let's now harmonize the format of all the dataframes. We want to end up with one dataframe with two colums: Name and Address

In [ ]:
column_names = ['Names', 'Adress']

In [ ]:
cnt = Counter()
for table in df_dict:
    cnt[str(len(df_dict[table].columns)) + ' columns'] += 1

cnt

There seem to be different formats for the dataframes. Let's focus on the most common for now (3, 4 and 5 colums).

In [ ]:
df_3 = pd.DataFrame(columns = [0,1,2])
df_4 = pd.DataFrame(columns = [0,1,2,3])
df_5 = pd.DataFrame(columns = [0,1,2,3,4])

for table in df_dict:
    if len(df_dict[table].columns) == 3: 
        df_3 = df_3.append(df_dict[table])
    elif len(df_dict[table].columns) == 4:
        df_4 = df_4.append(df_dict[table])
    elif len(df_dict[table].columns) == 5:
        df_5 = df_5.append(df_dict[table])

In [ ]:
df_3['Names'] = df_3[0] + ' ' + df_3[1]
df_3['Addresses'] = df_3[2]
df_3.drop(labels = [0,1,2], axis = 1, inplace = True)
df_3.reset_index(inplace = True)
df_3.head()

In [ ]:
df_4['Names'] = df_4[0] + df_4[1] + df_4[2]
df_4['Addresses'] = df_4[3]
df_4.drop(labels = [0,1,2,3], axis = 1, inplace = True)
df_4.reset_index(inplace = True)
df_4.head()

In [ ]:
df_4.Names

**Cleaning the addresses**

In [ ]:
# Text handling utilities
from string import punctuation
def lowercase_all(text):
    return text.lower()
def remove_punct(text):
    return ''.join([ch for ch in text if ch not in punctuation])

Spelling Corrector based on the work of Peter Norvig: http://norvig.com/spell-correct.html

In [ ]:
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('list_addresses.txt', encoding='utf-8').read())) 
#list_addresses.txt is a hand-corrected list of addresses

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if (w.isalpha() and w.lower() in WORDS))

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

We will use SpaCy to tokenize the addresses. We just need to add some rules to deal with special cases (like the hyphen in St-Honoré or commas at the end of a word).

In [ ]:
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load('fr_core_news_sm')

infix_re = re.compile(r'''[-~]''') #find hyphens
suffix_re = re.compile(r'''[,."']$''') #find , or . at end of word
def customize_tokenizer(nlp):
# Adds support to use `-` as the delimiter for tokenization
    return Tokenizer(nlp.vocab, 
                     infix_finditer=infix_re.finditer,
                     suffix_search=suffix_re.search, 
                     token_match=None)

nlp.tokenizer = customize_tokenizer(nlp)

We can now define some functions to clean the addresses. The first one corrects spelling error and the second one harmonizes all the results.

In [ ]:
def correct_adrs(adrs):
    clean_adrs = ''
    
    #Tokenize the address using SpaCy tokenizer
    adrs_parts = nlp(adrs)
    #print ([token.text for token in adrs_parts])
    
    #Find the street number
    last = 1
    if str(adrs_parts[-1]) in punctuation:
        number = adrs_parts[-2]
        last = 2
    else:
        number = adrs_parts[-1]
        
    #Correction of errors
    for i in range(len(adrs_parts)-last):
        if str(adrs_parts[i]) in punctuation:
            if str(adrs_parts[i]) == '-':
                clean_adrs = clean_adrs[:-1] + str(adrs_parts[i])
        else:
            clean_adrs += correction(str(adrs_parts[i])).capitalize()
            clean_adrs += ' '
            
    return clean_adrs + str(number)

convert_adrs = {'av':'Avenue', 
        'r':'Rue', 
        'bd':'Boulevard',
        'pl':'Place',
        'fr':'Faubourg'}

def clean_adrs(adrs):
    adrs = correct_adrs(adrs)
    adrs_part_punct = adrs.split()
    adrs_part = remove_punct(adrs).split()
    for i in range(len(adrs_part)):
        if lowercase_all(adrs_part[i]) in convert_adrs:
            adrs_part_punct[i] = convert_adrs[lowercase_all(adrs_part[i])]
    adrs = ' '.join(adrs_part_punct)
    return adrs

Let's have some tests (still need to get rid of NaN values)

In [ ]:
test_adrs = 'Ff S\'-llonoré. 21'
clean_adrs(test_adrs)

In [ ]:
for adr in df_dict['table_11'][2]:
        print(adr + ' : ' + clean_adrs(adr))

We can now clean the addresses in the whole dataframe (FIRST NEED TO HARMONIZE DATAFRAME FORMAT)

In [ ]:
'''
for table in df_dict:
    df_table[table]['Address'].apply(clean_adrs, inplace = True)
'''

**Cleaning the names**